In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test', 'train', 'train.csv', 'sample_submission.csv']
['028192187883168e2a7621c998dc447a.jpg', '95d2e16da6d3fb55b84f83f25a7dcf2e.jpg', '03fe79948e15465c62c400ad9d9fab13.jpg', 'f5fc5cebc8bf22c70240fd42bd3d64e8.jpg', 'd8d2f5becda9d56dadf2d64339395807.jpg', '39118ef96763296fdaac19b5445781ac.jpg', '5c5e5b595cd88af108eed5b9d5f3e0e6.jpg', 'f8738a9ee535240bd178101fe12fb3ba.jpg', 'c745412c3abc0bad208dcae6d0ab4e2f.jpg', '28118684e8726a7c503d45d6cc9aeb87.jpg', '00677f0440d465c2a685e33ded9bb729.jpg', '301d279597de4c878d7834a93ac0533b.jpg', '5407227a0ea553bf7b9afcab13568c99.jpg', '749d53fdf657eb1c3a3e5822326e861a.jpg', 'c0883b34f942d9f0dcae349fe8d9faa6.jpg', '6e70ab9ab2918211498fa7cdd577468d.jpg', '53f1507d109278d478d595f164a63ddf.jpg', 'c1a52218c4870a720e7f967c5474f5a7.jpg', '8e13a5612fcb302146b2474e3e6c82a3.jpg', '7b856932608e158f078edf2655fd6417.jpg', '7b7e1fa3406cc21027f9fddb100be4a6.jpg', 'a0dbee3b12d17fe597ec7a07a5e3f89b.jpg', '4d29e94b2083a9a7ebd46a78b7e5409b.jpg', 'e3c624396c9c78ce872a4d

In [6]:
import cv2
from keras.models import Sequential
from keras.layers import Flatten, Conv2D, MaxPool2D, Activation, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [3]:
df = pd.read_csv('../input/train.csv')
df.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [13]:
df['has_cactus'] = df['has_cactus'].astype(str)

train_generator = train_datagen.flow_from_dataframe(
    df,
    directory = '../input/train/train',
    subset = 'training',
    x_col = 'id',
    y_col = 'has_cactus',
    target_size = (32,32),
    class_mode = 'binary'
)
test_generator = train_datagen.flow_from_dataframe(
    df,
    directory = '../input/train/train',
    subset = 'validation',
    x_col = 'id',
    y_col = 'has_cactus',
    target_size = (32,32),
    class_mode = 'binary'
)

Found 14875 images belonging to 2 classes.
Found 2625 images belonging to 2 classes.


In [14]:
model = Sequential()

model.add(Conv2D(32, (3,3) ,activation = 'relu', input_shape = (32,32,3)))
model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPool2D(2,2))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))

In [15]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = Adam(), 
    metrics = ['accuracy']
)

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 2000,
    epochs = 10,
    validation_data = test_generator,
    validation_steps = 64
)

Epoch 1/10
 313/2000 [===>..........................] - ETA: 3:50 - loss: 0.2744 - acc: 0.8865

In [ ]:
ids = []
X_test = []

for image in os.listdir(test_directory):
    ids.append(image.split('.')[0])
    path = os.path.join(test_directory, image)
    X_test.append(cv2.imread(path))
    
X_test = np.array(X_test)
X_test = X_test.astype('float32') / 255

In [ ]:
predictions = model.predict(X_test)

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['has_cactus'] = predictions
submission['id'] = ids
submission.head(10)

In [ ]:
submission.to_csv('submission.csv', index = False)